In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

User = User[User['Nº cartão EVIO'] == 0]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)']]

print(User)

              Start date           Stop date  Duration (min)  \
31    02/25/2024 | 10:20  02/25/2024 | 10:37          16.500   
35    02/23/2024 | 08:37  02/23/2024 | 11:31         174.000   
52    02/20/2024 | 14:12  02/20/2024 | 20:23         370.850   
61    02/19/2024 | 08:31  02/19/2024 | 11:39         188.083   
91    02/11/2024 | 10:48  02/11/2024 | 11:14          25.983   
...                  ...                 ...             ...   
1536  04/03/2023 | 08:39  04/03/2023 | 13:38         298.967   
1542  03/31/2023 | 08:48  03/31/2023 | 13:36         288.500   
1545  03/31/2023 | 08:41  03/31/2023 | 10:26         105.350   
1548  03/31/2023 | 08:37  03/31/2023 | 08:45           8.283   
1559  03/29/2023 | 08:35  03/29/2023 | 10:26         111.117   

      Total Energy (kWh)  
31                 28.52  
35                 31.10  
52                 27.23  
61                 35.26  
91                 22.06  
...                  ...  
1536               43.80  
1542           

In [2]:
# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')


# Criando uma nova coluna 'Date' que contém apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Convertendo os valores de energia para o formato numérico adequado
#User['Total Energy (kWh)'] = User['Total Energy (kWh)'].str.replace(',', '.').astype(float)

# Agrupando os dados pela coluna 'Date' e somando os valores de 'Total Energy (kWh)'
User = User.groupby('Date')['Total Energy (kWh)'].sum().reset_index()

print(User)

# Criando um intervalo de datas que cubra todo o período de interesse
date_range = pd.date_range(start=User['Date'].min(), end=User['Date'].max(), freq='D')

# Criando um DataFrame com o intervalo de datas
date_df = pd.DataFrame({'Date': date_range})

# Renomeando a coluna 'Date' para 'Date_new'
date_df.rename(columns={'Date': 'Date_new'}, inplace=True)

# Convertendo a coluna 'Date' em 'daily_energy_sum' para datetime
User['Date'] = pd.to_datetime(User['Date'])

# Mesclando os DataFrames usando merge
merged_df = pd.merge(date_df, User, left_on='Date_new', right_on='Date', how='left')

# Preenchendo os valores ausentes na coluna 'Total Energy (kWh)' com zero
merged_df['Total Energy (kWh)'].fillna(0, inplace=True)
merged_df = merged_df.drop(columns=['Date'])
merged_df.rename(columns={'Date_new': 'Date'}, inplace=True)

print(merged_df)

           Date  Total Energy (kWh)
0    2023-03-29               12.60
1    2023-03-31               61.66
2    2023-04-03               43.80
3    2023-04-05               51.27
4    2023-04-06               29.33
..          ...                 ...
107  2024-02-11               22.06
108  2024-02-19               35.26
109  2024-02-20               27.23
110  2024-02-23               31.10
111  2024-02-25               28.52

[112 rows x 2 columns]
          Date  Total Energy (kWh)
0   2023-03-29               12.60
1   2023-03-30                0.00
2   2023-03-31               61.66
3   2023-04-01                0.00
4   2023-04-02                0.00
..         ...                 ...
329 2024-02-21                0.00
330 2024-02-22                0.00
331 2024-02-23               31.10
332 2024-02-24                0.00
333 2024-02-25               28.52

[334 rows x 2 columns]


In [3]:
merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date  Total Energy (kWh)  Weekday
0   2023-03-29               12.60        3
1   2023-03-30                0.00        4
2   2023-03-31               61.66        5
3   2023-04-01                0.00        6
4   2023-04-02                0.00        7
..         ...                 ...      ...
329 2024-02-21                0.00        3
330 2024-02-22                0.00        4
331 2024-02-23               31.10        5
332 2024-02-24                0.00        6
333 2024-02-25               28.52        7

[334 rows x 3 columns]


In [4]:
# Adicionar a coluna 'Carregar'
merged_df['Carregar'] = merged_df['Total Energy (kWh)'] > 0

# Visualizar as primeiras linhas do DataFrame
print(merged_df)

# Criar uma coluna 'Last_Charge_Date' que desloca a data do último carregamento
merged_df['Last_Charge_Date'] = merged_df['Date'].where(merged_df['Carregar']).ffill()

# Calcular os dias desde o último carregamento
merged_df['Days_since_last_charge'] = (merged_df['Date'] - merged_df['Last_Charge_Date']).dt.days

# Preencher valores NaN em 'Days_since_last_charge' com 0 (caso não tenha ocorrido carregamento antes)
merged_df['Days_since_last_charge'] = merged_df['Days_since_last_charge'].fillna(0).astype(int)

# Visualizar o DataFrame com a nova coluna
print(merged_df)

          Date  Total Energy (kWh)  Weekday  Carregar
0   2023-03-29               12.60        3      True
1   2023-03-30                0.00        4     False
2   2023-03-31               61.66        5      True
3   2023-04-01                0.00        6     False
4   2023-04-02                0.00        7     False
..         ...                 ...      ...       ...
329 2024-02-21                0.00        3     False
330 2024-02-22                0.00        4     False
331 2024-02-23               31.10        5      True
332 2024-02-24                0.00        6     False
333 2024-02-25               28.52        7      True

[334 rows x 4 columns]
          Date  Total Energy (kWh)  Weekday  Carregar Last_Charge_Date  \
0   2023-03-29               12.60        3      True       2023-03-29   
1   2023-03-30                0.00        4     False       2023-03-29   
2   2023-03-31               61.66        5      True       2023-03-31   
3   2023-04-01                0.

In [5]:
# Adicionar a coluna 'Carregar'
merged_df['Carregar'] = merged_df['Total Energy (kWh)'] > 0

# Visualizar as primeiras linhas do DataFrame
print(merged_df)

# Criar uma coluna 'Last_Charge_Date' que desloca a data do último carregamento
merged_df['Last_Charge_Date'] = merged_df['Date'].where(merged_df['Carregar']).shift(1).ffill()

# Visualizar o DataFrame com a nova coluna
print(merged_df)

          Date  Total Energy (kWh)  Weekday  Carregar Last_Charge_Date  \
0   2023-03-29               12.60        3      True       2023-03-29   
1   2023-03-30                0.00        4     False       2023-03-29   
2   2023-03-31               61.66        5      True       2023-03-31   
3   2023-04-01                0.00        6     False       2023-03-31   
4   2023-04-02                0.00        7     False       2023-03-31   
..         ...                 ...      ...       ...              ...   
329 2024-02-21                0.00        3     False       2024-02-20   
330 2024-02-22                0.00        4     False       2024-02-20   
331 2024-02-23               31.10        5      True       2024-02-23   
332 2024-02-24                0.00        6     False       2024-02-23   
333 2024-02-25               28.52        7      True       2024-02-25   

     Days_since_last_charge  
0                         0  
1                         1  
2                    

In [6]:
# Calcular os dias desde o último carregamento
merged_df['Days_since_last_charge'] = (merged_df['Date'] - merged_df['Last_Charge_Date']).dt.days

# Preencher valores NaN em 'Days_since_last_charge' com 0 (caso não tenha ocorrido carregamento antes)
merged_df['Days_since_last_charge'] = merged_df['Days_since_last_charge'].fillna(0).astype(int)

# Visualizar o DataFrame com a nova coluna
print(merged_df)

          Date  Total Energy (kWh)  Weekday  Carregar Last_Charge_Date  \
0   2023-03-29               12.60        3      True              NaT   
1   2023-03-30                0.00        4     False       2023-03-29   
2   2023-03-31               61.66        5      True       2023-03-29   
3   2023-04-01                0.00        6     False       2023-03-31   
4   2023-04-02                0.00        7     False       2023-03-31   
..         ...                 ...      ...       ...              ...   
329 2024-02-21                0.00        3     False       2024-02-20   
330 2024-02-22                0.00        4     False       2024-02-20   
331 2024-02-23               31.10        5      True       2024-02-20   
332 2024-02-24                0.00        6     False       2024-02-23   
333 2024-02-25               28.52        7      True       2024-02-23   

     Days_since_last_charge  
0                         0  
1                         1  
2                    

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Features e rótulo
#X = merged_df[['Weekday']].values
X = merged_df[['Weekday', 'Days_since_last_charge']].values
y = merged_df['Carregar'].astype(int).values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42, shuffle=True)

# Treinar o modelo de regressão logística
model = LogisticRegression()
model.fit(X_train, y_train)


print(X_test)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

print(y_test, predictions)

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print("Matriz de confusão:")
print(conf_matrix)

[[ 7  7]
 [ 4  2]
 [ 6 55]
 [ 2  3]
 [ 4 39]
 [ 5  2]
 [ 4  2]
 [ 1  1]
 [ 4  2]
 [ 4  2]
 [ 1 57]
 [ 5  1]
 [ 1  1]
 [ 3  1]
 [ 2  5]
 [ 2  1]
 [ 5  1]
 [ 2  8]
 [ 2 58]
 [ 5  1]
 [ 7  1]
 [ 7  1]
 [ 4  3]
 [ 6  1]
 [ 1  3]
 [ 1  2]
 [ 5  1]
 [ 5  5]
 [ 5  1]
 [ 4  4]
 [ 3  1]
 [ 3  7]
 [ 3 45]
 [ 3  4]
 [ 1  2]
 [ 4  6]
 [ 3  1]
 [ 4  6]
 [ 7 28]
 [ 2  1]
 [ 1 15]
 [ 3 24]
 [ 7  5]
 [ 2  2]
 [ 4  2]
 [ 5 54]
 [ 4  2]
 [ 6 27]
 [ 6  1]
 [ 7  1]
 [ 6  1]
 [ 1  1]
 [ 6  1]
 [ 5  2]
 [ 6  1]
 [ 6  1]
 [ 3  2]
 [ 2 37]
 [ 3  9]
 [ 7  5]
 [ 6  3]
 [ 5  4]
 [ 3  1]
 [ 6  1]
 [ 3 59]
 [ 6  2]
 [ 7 42]
 [ 3  1]
 [ 4 60]
 [ 4  1]
 [ 6  3]
 [ 6  6]
 [ 1  3]
 [ 7  7]
 [ 3  1]
 [ 5  3]
 [ 1  2]
 [ 2  1]
 [ 6  5]
 [ 5  1]
 [ 7  2]
 [ 3  2]
 [ 4  2]
 [ 2  1]
 [ 2  1]
 [ 2  1]
 [ 6 13]
 [ 2  6]
 [ 6  2]
 [ 2  1]
 [ 6  3]
 [ 5  3]
 [ 1  7]
 [ 6 41]
 [ 3  2]
 [ 4  1]
 [ 2  1]
 [ 2  1]
 [ 5  1]
 [ 5 12]
 [ 4  4]
 [ 7  2]
 [ 3  2]
 [ 3 38]
 [ 3  1]
 [ 6  2]
 [ 4  1]
 [ 6  1]
 [ 7  1]
 [ 2 15]
 [ 7 21]
 

In [8]:
from sklearn.linear_model import LinearRegression

# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão linear
reg_model = LinearRegression()
reg_model.fit(X_reg, y_reg)

# Fazer previsões (usando X_test filtrado para onde a previsão foi 1)
charging_indices = [i for i, pred in enumerate(predictions) if pred == 1]
X_test_charging = X_test[charging_indices]

# Prever a quantidade de energia carregada
energy_predictions = reg_model.predict(X_test_charging)

print("Previsões de energia carregada:", energy_predictions)

Previsões de energia carregada: [32.05828068 31.92613656 31.92613656 32.22702925 32.07658291 31.90783433
 31.91698544 32.22702925 31.91698544 32.22702925 32.07658291 32.06743179
 31.92613656 32.21787814 32.22702925 32.22702925 32.3774756  31.90783433
 32.22702925 31.91698544 32.07658291 32.21787814 32.07658291 32.07658291
 32.07658291 32.07658291 32.21787814 32.3774756  32.07658291 32.07658291
 32.21787814 32.22702925 32.3774756  32.07658291 32.3774756  31.90783433
 32.22702925 32.22702925 32.07658291 31.92613656 32.22702925 32.3774756
 31.91698544 31.90783433 32.07658291 32.3774756  32.07658291 31.91698544
 32.07658291 32.22702925 32.22702925 32.07658291 31.91698544 31.92613656
 32.07658291 32.3774756  32.05828068 32.3774756  32.05828068 32.21787814
 32.3774756  31.91698544 32.3774756  31.90783433 31.89868321 32.07658291
 31.91698544 32.3774756  32.21787814 32.23618037 31.90783433 32.22702925
 32.21787814 32.06743179 31.90783433 31.91698544 31.92613656 32.3774756
 32.22702925 31.90783

In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Supondo que merged_df é o DataFrame com os dados fornecidos e já possui as colunas necessárias

# Features e rótulo para a previsão binária
X = merged_df[['Weekday', 'Days_since_last_charge']].values
y = merged_df['Carregar'].astype(int).values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Treinar o modelo de regressão logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = logistic_model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print("Matriz de confusão:")
print(conf_matrix)

# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão linear
reg_model = LinearRegression()
reg_model.fit(X_reg, y_reg)

# Fazer previsões (usando X_test completo)
energy_predictions = np.zeros(len(X_test))  # Inicializar com zeros
charging_indices = [i for i, pred in enumerate(predictions) if pred == 1]

if len(charging_indices) > 0:
    X_test_charging = X_test[charging_indices]
    energy_predictions_charging = reg_model.predict(X_test_charging)
    for idx, val in zip(charging_indices, energy_predictions_charging):
        energy_predictions[idx] = val

# Consolidação dos resultados
result_df = pd.DataFrame({
    'Date': merged_df.iloc[X_test[:, 0].astype(int)].index,
    'Weekday': X_test[:, 0],
    'Days_since_last_charge': X_test[:, 1],
    'Carregar (Previsão)': predictions,
    'Total Energy (Previsão)': energy_predictions
})

result_df['Carregar (Previsão)'] = result_df['Carregar (Previsão)'].map({0: 'Não Carregar', 1: 'Carregar'})

print("Previsões finais:")
print(result_df)

Acurácia do modelo: 0.6716417910447762
Matriz de confusão:
[[42  6]
 [16  3]]
Previsões finais:
    Date  Weekday  Days_since_last_charge Carregar (Previsão)  \
0      7        7                       7        Não Carregar   
1      4        4                       2        Não Carregar   
2      6        6                      55        Não Carregar   
3      2        2                       3        Não Carregar   
4      4        4                      39        Não Carregar   
..   ...      ...                     ...                 ...   
62     3        3                       1        Não Carregar   
63     6        6                       1        Não Carregar   
64     3        3                      59        Não Carregar   
65     6        6                       2        Não Carregar   
66     7        7                      42        Não Carregar   

    Total Energy (Previsão)  
0                       0.0  
1                       0.0  
2                       0.0  
3  

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# Supondo que merged_df é o DataFrame com os dados fornecidos e já possui as colunas necessárias
# Adicionar uma coluna 'Carregar' baseada em 'Total Energy (kWh)'
merged_df['Carregar'] = (merged_df['Total Energy (kWh)'] > 0).astype(int)

# Features e rótulo para a previsão binária
X = merged_df[['Weekday', 'Days_since_last_charge']].values
y = merged_df['Carregar'].astype(int).values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Treinar o modelo de regressão logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = logistic_model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print("Matriz de confusão:")
print(conf_matrix)

# Identificar índices dos true positives no conjunto de teste
true_positive_indices = np.where((y_test == 1) & (predictions == 1))[0]


# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

print(charging_data)

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão linear
#reg_model = LinearRegression()
reg_model = GradientBoostingRegressor()
reg_model.fit(X_reg, y_reg)

# Fazer previsões (usando X_test completo)
energy_predictions = np.zeros(len(X_test))  # Inicializar com zeros
charging_indices = [i for i, pred in enumerate(predictions) if pred == 1]

if len(charging_indices) > 0:
    X_test_charging = X_test[charging_indices]
    energy_predictions_charging = reg_model.predict(X_test_charging)
    for idx, val in zip(charging_indices, energy_predictions_charging):
        energy_predictions[idx] = val

# Consolidação dos resultados com datas corretas
test_indices = X_test[:, 0].astype(int)
result_df = pd.DataFrame({
    'Date': merged_df.iloc[test_indices].index,
    'Weekday': X_test[:, 0],
    'Days_since_last_charge': X_test[:, 1],
    'Carregar (Previsão)': predictions,
    'Total Energy (Previsão)': energy_predictions
})

result_df['Carregar (Previsão)'] = result_df['Carregar (Previsão)'].map({0: 'Não Carregar', 1: 'Carregar'})

print("Previsões finais:")
print(result_df)

# Obter as linhas correspondentes aos true positives
true_positive_rows = merged_df.iloc[true_positive_indices]
true_positive_rows['Total Energy (Previsão)'] = reg_model.predict(true_positive_rows[['Weekday', 'Days_since_last_charge']].values)

print("Linhas dos True Positives:")
print(true_positive_rows)

Acurácia do modelo: 0.6716417910447762
Matriz de confusão:
[[42  6]
 [16  3]]
          Date  Total Energy (kWh)  Weekday  Carregar Last_Charge_Date  \
0   2023-03-29               12.60        3         1              NaT   
2   2023-03-31               61.66        5         1       2023-03-29   
5   2023-04-03               43.80        1         1       2023-03-31   
7   2023-04-05               51.27        3         1       2023-04-03   
8   2023-04-06               29.33        4         1       2023-04-05   
..         ...                 ...      ...       ...              ...   
319 2024-02-11               22.06        7         1       2024-02-05   
327 2024-02-19               35.26        1         1       2024-02-11   
328 2024-02-20               27.23        2         1       2024-02-19   
331 2024-02-23               31.10        5         1       2024-02-20   
333 2024-02-25               28.52        7         1       2024-02-23   

     Days_since_last_charge  
0  

C:\Users\miguel.preto\AppData\Local\Temp\ipykernel_20512\513728761.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_positive_rows['Total Energy (Previsão)'] = reg_model.predict(true_positive_rows[['Weekday', 'Days_since_last_charge']].values)


In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Supondo que merged_df é o DataFrame com os dados fornecidos e já possui as colunas necessárias
# Adicionar uma coluna 'Carregar' baseada em 'Total Energy (kWh)'
merged_df['Carregar'] = (merged_df['Total Energy (kWh)'] > 0).astype(int)

# Filtrar dados onde houve carregamento
charging_data = merged_df[merged_df['Carregar'] == 1]

# Features para o modelo de regressão
X_reg = charging_data[['Weekday', 'Days_since_last_charge']].values
y_reg = charging_data['Total Energy (kWh)'].values

# Treinar o modelo de regressão usando Gradient Boosting Regressor
reg_model = GradientBoostingRegressor()
reg_model.fit(X_reg, y_reg)

# Fazer previsões para todos os dados de charging_data
energy_predictions = reg_model.predict(X_reg)

# Calcular o RMSE usando as previsões e os valores reais de 'Total Energy (kWh)'
rmse = mean_squared_error(y_reg, energy_predictions, squared=False)
print(f"RMSE do modelo de regressão para quantidade de energia: {rmse:.2f} kWh")

# Consolidação dos resultados com datas corretas
result_df = pd.DataFrame({
    'Date': charging_data['Date'],
    'Weekday': charging_data['Weekday'],
    'Days_since_last_charge': charging_data['Days_since_last_charge'],
    'Carregar (Previsão)': charging_data['Carregar'].map({0: 'Não Carregar', 1: 'Carregar'}),
    'Total Energy (Previsão)': energy_predictions
})

print("\nPrevisões finais:")
print(result_df)


RMSE do modelo de regressão para quantidade de energia: 9.69 kWh

Previsões finais:
          Date  Weekday  Days_since_last_charge Carregar (Previsão)  \
0   2023-03-29        3                       0            Carregar   
2   2023-03-31        5                       1            Carregar   
5   2023-04-03        1                       2            Carregar   
7   2023-04-05        3                       1            Carregar   
8   2023-04-06        4                       0            Carregar   
..         ...      ...                     ...                 ...   
319 2024-02-11        7                       5            Carregar   
327 2024-02-19        1                       7            Carregar   
328 2024-02-20        2                       0            Carregar   
331 2024-02-23        5                       2            Carregar   
333 2024-02-25        7                       1            Carregar   

     Total Energy (Previsão)  
0                  26.241364  
2

C:\Users\miguel.preto\Anaconda3\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
